In [9]:
from selenium import webdriver                 
from selenium.webdriver.common.by import By   
from webdriver_manager.chrome import ChromeDriverManager
import time             
from bs4 import BeautifulSoup 
import pandas as pd        
import re      

## Virginia Laws

In [70]:
df = pd.read_csv('./virginia_laws.csv')
df['law_text'] = None
df['location'] = 'VA'

In [ ]:
for i in range(len(df)):
    all_content = []
    #get the url - stopped at i = 42
    url = df.loc[i, 'Law']
    driver = webdriver.Chrome()
    #go to url
    driver.get(url)
    try:
        element = driver.find_element(By.CLASS_NAME, "content")
        print('got the element')
        p_elements = element.find_elements(By.TAG_NAME, 'p')
        for p_element in p_elements:
            all_content.append(p_element.text)
        str_content = ''
        for para in all_content:
            str_content = str_content + para
        df.loc[i, 'law_text'] = str_content
    except Exception as e:
        print('data not pulled for', i)
    finally:
        driver.quit()
        print('finished law number: ' , i)

In [ ]:
def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()
df['law_text'] = df['law_text'].apply(clean_text)
#probably should have done in selenium, but column of the law code name
def get_lawname(text):
    text = text.split('/')[-4:-1]
    law_up = ''
    for i in text:
        output = re.search(r"([a-z]+)([0-9.-]+)", i)
        law_up = law_up + output.group(1) + ' ' + output.group(2) + ' '
    return law_up
df['law_title'] = df['Law'].apply(get_lawname)

In [125]:
#save to csv
df.to_csv('./virginia_laws.csv', index = False)

# DC laws

In [115]:
df_2 = pd.read_csv('./dc_laws.csv')
df_2['law_text'] = None
df_2['location'] = 'DC'

In [130]:
def using_selenium(df, class_tag, title, tag_name):
    for i in range(len(df)):
        all_content = []
        #get the url - stopped at i = 42
        url = df.loc[i, 'Law']
        driver = webdriver.Chrome()
        #go to url
        driver.get(url)
        try:
            element = driver.find_element(By.CLASS_NAME, class_tag)
            print('got the element')
            title = element.find_element(By.TAG_NAME, title)
            df.loc[i, 'law_title'] = title.text
            print('got the title')
            p_elements = element.find_elements(By.TAG_NAME, tag_name)
            print('got the paragraphs')
            for p_element in p_elements:
                all_content.append(p_element.text)
            str_content = ''
            for para in all_content:
                str_content = str_content + para
            df.loc[i, 'law_text'] = str_content
        except Exception as e:
            print('data not pulled for', i)
        finally:
            driver.quit()
            print('finished law number: ' , i)



# using_selenium(df_2, 'content', 'h1', 'p')

In [127]:
#save to csv
df_2.to_csv('./dc_laws.csv', index = False)

## Maryland Data

In [128]:
df_3 = pd.read_csv('./maryland_laws.csv')
df_3['law_text'] = None
df_3['law_title'] = None
df_3['location'] = 'MD'

In [143]:
def using_selenium(df):
    for i in range(len(df)):
        #get the url - stopped at i = 42
        url = df.loc[i, 'Law']
        driver = webdriver.Chrome()
        #go to url
        driver.get(url)
        try:
            element = driver.find_element(By.ID, 'StatuteText')
            df.loc[i, 'law_text'] = element.text
        except Exception as e:
            print('data not pulled for', i)
        finally:
            driver.quit()
            print('finished law number: ' , i)

In [144]:
using_selenium(df_3)

finished law number:  0
finished law number:  1
finished law number:  2
finished law number:  3
finished law number:  4
finished law number:  5
finished law number:  6
finished law number:  7
finished law number:  8
finished law number:  9
finished law number:  10
finished law number:  11
finished law number:  12
finished law number:  13
finished law number:  14
finished law number:  15
finished law number:  16
finished law number:  17
finished law number:  18
finished law number:  19
finished law number:  20
finished law number:  21
finished law number:  22
finished law number:  23
finished law number:  24
finished law number:  25
finished law number:  26
finished law number:  27
finished law number:  28
finished law number:  29
finished law number:  30
finished law number:  31
finished law number:  32
finished law number:  33
finished law number:  34
finished law number:  35
finished law number:  36
finished law number:  37
finished law number:  38
finished law number:  39
finished l

In [190]:
text = df_3['law_text'][47]
text = text.replace('\n\n\n', '\n\n')
text = text.split('\n\n')
text

['Article - Real Property',
 'Previous\nNext',
 '§8–304.',
 '    (a)    When an action of distress is filed, the clerk shall issue an order directing the defendant to appear and show cause at a stated time why levy under an action of distress should not be made. The hearing may be not earlier than seven days from date of service on the defendant.',
 '    (b)    In addition, the order shall:',
 '        (1)    Direct the time within which service of the petition and show–cause order shall be made on the defendant; and',
 '        (2)    Inform the defendant that (i) the defendant may appear at the time stated and present evidence on the defendant’s behalf; and (ii) if the defendant fails to appear, all goods on the leased premises not exempted by law may be levied on and removed by the sheriff.',
 'Previous\nNext']

In [191]:
def clean_maryland(text):
    if isinstance(text, str):
        text = text.replace('\n\n\n', '\n\n')
        text = text.split('\n\n')
        
        # Check if text has enough elements
        if len(text) > 3:
            title = text[2].replace('\n§', '').replace('.', '')
            title = f"Article {title}"
            text = text[3].strip()
            return title, text
        else:
            # Handle the case when text does not have enough elements
            return None, None
    else:
        # Handle the case when text is not a string
        return None, None

# Apply the function to the DataFrame
df_3['law_title'], df_3['law_text'] = zip(*df_3['law_text'].apply(clean_maryland))


In [197]:
df_3[df_3['law_text'].isnull()]

,Law,Keywords,Type,law_text,law_title,location
134,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,NaN,NaN,None,None,MD


In [200]:
df_3[:135]

,Law,Keywords,Type,law_text,law_title,location
0,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,"Transferee, Reversion, Leased Property, Rent, ...",NaN,A transferee of the reversion in leased proper...,Article §8–101,MD
1,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,NaN,NaN,If the reversion of any leased premises merges...,Article §8–102,MD
2,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,NaN,NaN,There is no merger by reason of any grant by w...,Article §8–103,MD
3,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,NaN,NaN,Any grant of a nonpossessory corporeal estate ...,Article §8–104,MD
4,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,NaN,NaN,If the effect of any provision of a lease is t...,Article §8–105,MD
...,...,...,...,...,...,...
130,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,NaN,NaN,(a) A park owner shall offer all current an...,Article §8A–202,MD
131,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,NaN,NaN,(a) (1) A park owner shall establish rea...,Article §8A–301,MD
132,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,NaN,NaN,(a) A park owner may increase a park fee on...,Article §8A–401,MD
133,https://mgaleg.maryland.gov/mgawebsite/Laws/St...,NaN,NaN,(a) An entrance or exit fee is prohibited.,Article §8A–402,MD


In [203]:
df_3.loc[134,'law_title'] = 'Article §8-403'
df_3.loc[134,'law_text'] = '(a)    A park owner may charge the resident a reasonable service fee, based on an amount that the park owner directly incurs, for installing, placing on, or removing a mobile home from the site. (b)    In each case where a fee has been charged by a park owner, a written description detailing the fee shall be provided by the park owner to the resident.'


In [204]:
df_3[df_3['law_text'].isnull()]

,Law,Keywords,Type,law_text,law_title,location


In [205]:
df_3.to_csv('./maryland_laws.csv', index = False)